In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

In [2]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
import json
from pathlib import Path


CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
bucket = os.environ["BUCKET"]
role = os.environ["ROLE"]
S3_LOCATION = f"s3://{bucket}/football"

In [5]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

LOCAL_MODE = True

pipeline_session = PipelineSession(default_bucket=bucket) if not LOCAL_MODE else None

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=bucket),
        # "instance_type": "local",
        "instance_type": "ml.m5.xlarge",
        "image": None,
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.m5.xlarge",
        "image": None,
    }

# config["framework_version"] = "2.11"
config["py_version"] = "py312"

Windows Support for Local Mode is Experimental


In [6]:
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

In [7]:
from sagemaker.s3 import S3Uploader

df_local_path=str(os.environ['DATA_FILEPATH_X']) 
y_local_path=str(os.environ['DATA_FILEPATH_y'])

# S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)
# S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)

In [8]:
# 's3://football-data-kamil/football/data/df.csv'
# 's3://football-data-kamil/football/data/y.csv'

In [9]:
import pandas as pd
import numpy as np

football = pd.read_csv(df_local_path)
y = pd.read_csv(y_local_path)
football.head()

,player_rating_home_player_1,player_rating_home_player_2,player_rating_home_player_3,player_rating_home_player_4,player_rating_home_player_5,player_rating_home_player_6,player_rating_home_player_7,player_rating_home_player_8,player_rating_home_player_9,player_rating_home_player_10,...,ewm_possession_home,ewm_possession_away,avg_home_rating_attack,avg_away_rating_attack,avg_away_rating_defence,avg_home_rating_defence,average_rating_home,average_rating_away,defensive_weakness_home,defensive_weakness_away
0,72,72,72,72,72,72,79,75,85,87,...,NaN,NaN,78.50,71.50,71.0,72.0,75.545455,71.272727,NaN,NaN
1,79,79,79,79,79,79,78,75,74,81,...,NaN,NaN,75.83,82.33,84.0,79.0,77.272727,83.090909,NaN,NaN
2,77,77,77,77,77,77,72,82,76,73,...,NaN,NaN,76.33,73.50,76.0,77.0,76.636364,74.636364,NaN,NaN
3,82,82,82,82,82,82,72,73,77,73,...,NaN,NaN,76.33,77.50,73.0,82.0,78.909091,75.454545,NaN,NaN
4,77,77,77,77,77,77,67,75,82,84,...,NaN,NaN,77.00,79.33,84.0,77.0,77.000000,81.454545,NaN,NaN


In [10]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [11]:
from sagemaker.workflow.parameters import ParameterString

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

In [12]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    base_job_name="preprocess-data",
    framework_version="1.2-1",
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

In [13]:
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput


preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(
                source=dataset_location, destination="/opt/ml/processing/input"
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source="/opt/ml/processing/train",
                destination=f"{S3_LOCATION}/preprocessing/train",
            ),
            ProcessingOutput(
                output_name="validation",
                source="/opt/ml/processing/validation",
                destination=f"{S3_LOCATION}/preprocessing/validation",
            ),
            ProcessingOutput(
                output_name="test",
                source="/opt/ml/processing/test",
                destination=f"{S3_LOCATION}/preprocessing/test",
            ),
            ProcessingOutput(
                output_name="model",
                source="/opt/ml/processing/model",
                destination=f"{S3_LOCATION}/preprocessing/model",
            ),
            ProcessingOutput(
                output_name="train-baseline",
                source="/opt/ml/processing/train-baseline",
                destination=f"{S3_LOCATION}/preprocessing/train-baseline",
            ),
            ProcessingOutput(
                output_name="test-baseline",
                source="/opt/ml/processing/test-baseline",
                destination=f"{S3_LOCATION}/preprocessing/test-baseline",
            ),
        ],
    ),
    cache_config=cache_config,
)

In [17]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

session1_pipeline = Pipeline(
    name="session1-pipeline",
    parameters=[dataset_location],
    steps=[
        preprocessing_step,
    ],
    # pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session1_pipeline.upsert(role_arn=role)

{'PipelineArn': 'session1-pipeline'}

In [25]:
# %%script false --no-raise-error
#| eval: false
#| code: true
#| output: false

session1_pipeline.start()

KeyboardInterrupt: 